1. [Імпорт бібліотек](#Імпорт-необхідних-бібліотек)

2. [Завантаження даних](#За?вантаження-даних)

3. [Предиктори та цільова змінна](#Визначаємо-предиктори-та-цільову-змінну)

4. [Основні функції](#Цільова-функція-помилки)

5. [Результати](#Результати-оптимізації-гіперпараметрів)

### Імпорт необхідних бібліотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
import Utilities
import Hive
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

#### Завантаження даних

In [2]:
data = pd.read_csv('kc_house_data.csv')

#### Розмірність даних

In [3]:
data.shape

(21613, 21)

#### Перші 3 рядки в наборі даних (їх вигляд)

In [4]:
data.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


#### Визначаємо предиктори та цільову змінну

In [5]:
features = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated'] # предиктори

target = 'price' # цільова змінна

#### Поділ даних на тренувальну та тестувальну вибірку (у відношенні 80\20)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], random_state=42)

#### Нормалізація та стандартизація даних

In [7]:
ss = StandardScaler()
ss.fit(data[features])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [8]:
new_X_train = ss.transform(X_train)
new_X_test = ss.transform(X_test)

#### Цільова функція помилки, яку потрібно мінімізувати

In [9]:
def evaluator(weights):
    
    #n = X_train.loc[:10, :].shape[0]
    n = 100
    target_val = y_train.loc[:100]
    
    return (1/n) * sum([pow(np.dot(weights, 
                                new_X_train[i]) - target_val.iloc[i], 2)
                        for i in range(n)])

#### Функція для "прогонки" бджолиного алгоритму

In [10]:
def run():

    # створюємо модель рою пчіл (об'єкт класу BeeHive)
    ndim = len(features)
    model = Hive.BeeHive(lower     = [0.0015] *ndim ,
                         upper     = [1000]*ndim ,
                         fun       = evaluator ,
                         numb_bees =  50       ,
                         max_itrs  =  50       ,)

    # навчаємо модель
    cost = model.run()

    return model

#### Функція для виконання прогнозу

In [11]:
make_prediction = lambda x, w: np.dot(x, w)

#### Записуємо результат оптимізації параметрів лінійної регресії бджолиним алгоритмом

In [12]:
res = run() # змінна, що містить результат роботи бджолиного алгоритму

### Результати оптимізації гіперпараметрів

#### "Важливість предикторів" (коефіцієнт при предикторі показує його степінь важливості при прогнозуванні цільової змінної)

In [13]:
dict(zip(features, res.solution))

{'bathrooms': 1000,
 'bedrooms': 1000,
 'condition': 1000,
 'floors': 1000,
 'grade': 1000,
 'sqft_above': 1000,
 'sqft_basement': 1000,
 'sqft_living': 1000,
 'sqft_lot': 0.0015,
 'view': 1000,
 'waterfront': 0.0015,
 'yr_built': 1000,
 'yr_renovated': 0.0015}

### Середньоквадратична помилка результів методом бджілки та лінійною регресією

#### Бджолиний алгоритм

In [20]:
bee = mean_squared_error(y_train, make_prediction(X_train, res.solution))
bee

338882.96714173106

#### Навчаємо модель лінійної регресії за допомогою пакета для машинного навчання - scikit-learn

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [26]:
lrer = mean_squared_error(y_train,lr.predict(X_train))
lrer

45423.0